# Data preparation and processing notebook

Dataset descriptions and references: 

1) Yamagishi R, Kaneko H. Data from comprehensive analysis of nuclear localization signals. Data Brief. 2015 Dec 12;6:200-3. doi: 10.1016/j.dib.2015.11.064. PMID: 26862559; PMCID: PMC4707185.

    UniProt IDs were extracted from a spreadsheet containing documented NLS sequences and the proteins in which they are located inside the supplementary materials of the paper cited above. These UniProt IDs were entered in the ID-mapping query on the UniProt website, which then generated a FASTA file containing all the full sequences of the proteins. We then used the SeqIO method inside the Biopython package to extract the sequences from the FASTA file and add them to a CSV. This CSV containing the full sequences was then merged with Pandas such that the proteins, organized by UniProt ID, contained both the NLS sequences and the full sequence.

2)  https://services.healthtech.dtu.dk/services/DeepLoc-2.0/#




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from data_prep import *

In [2]:
df = pd.read_csv("../data/SortingSignalsSwissprot.csv")
df["Types"].unique()

array(['MT', 'SP_GPI', 'GPI', 'SP_TM', 'SP', 'NLS', 'TH', 'CH', 'TM',
       'PTS', 'NES', 'NLS_NES', 'TM_MT', nan, 'SP_TM_PTS', 'SP_PTS'],
      dtype=object)

In [3]:
# Drop rows with specific values in the 'Types' column
values_to_drop = ['NLS_NES', 'TM_MT', np.nan, 'SP_TM_PTS', 'SP_PTS', 'SP_GPI', 'SP_TM']
df = df[~df['Types'].isin(values_to_drop)]

In [4]:
df = df.drop(columns="Kingdom")

In [5]:
print(df["Types"].unique())

type_counts = df['Types'].value_counts()
print(type_counts)

['MT' 'GPI' 'SP' 'NLS' 'TH' 'CH' 'TM' 'PTS' 'NES']
Types
SP     768
MT     236
NLS    131
PTS    125
GPI     92
CH      90
NES     83
TH      42
TM      35
Name: count, dtype: int64


In [6]:
#renaming columns before combining two dataframes 

df2 = pd.read_csv("../data/finalized_complete_NLS_sequence_table.csv")

df2 = df2.rename(columns={'Sequence_full': "Sequence"})

df2["Types"] = "NLS"

df2 = df2.rename(columns={"UniProt ID": "ACC"})

df2['AnnotEncoded'] = df2.apply(generate_annotation, axis=1)

df2 = df2.drop(columns=['Name', 'Begin','End','Length', 'Evidence', 'Sequence_nls', 'ECO code'])

In [7]:
#Combining the two data frames
stacked_df = pd.concat([df, df2], ignore_index=True)

In [8]:
# Function to remove rows containing "B", "U", or "X" in the sequence
cleaned_stacked_df = remove_bux(stacked_df)

In [9]:
#Adding the length column which shows lengths of the full protein sequences
cleaned_stacked_df["Length"] = cleaned_stacked_df["Sequence"].apply(lambda x: len(x))

In [10]:
# removing duplicates 
df_cleaned = cleaned_stacked_df.drop_duplicates(keep=False)

In [14]:
#cleaned_modified_df2.to_csv("cleaned_modified_df2.csv",index=False)

df_cleaned.to_csv("../data/finalized_df_cleaned.csv",index=False)